In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import numpy as np
import math

## z-score и t-score

### Поиск z-score по площади и площади по z-score

In [2]:
# поиск площади слева от z-score, например при z-score = 1.96
print(st.norm.cdf(1.96)) # должно получиться 0.975
# если взять -1.96
print(st.norm.cdf(-1.96)) # должно получиться 0.025 потому что колокол симметричен

# если мы хотим найти площадь справа от 1.96 (правый хвост, например при тестированни правосторонней гипотезы)
print(1 - st.norm.cdf(1.96))

# площадь между -1.96 и 1.96, из большей площади вычитаем меньшую
print(st.norm.cdf(1.96) - st.norm.cdf(-1.96))

0.9750021048517795
0.024997895148220435
0.024997895148220484
0.950004209703559


In [3]:
# обратный процесс, поиск z-score по площади (например, 97.5% площади слева от z-score, которое мы ищем)
print(st.norm.ppf(0.975))

# показать что процессы взаимообратны можно так (погрешность связана с округлением)
print(st.norm.ppf(st.norm.cdf(1.96)))

1.959963984540054
1.959999999999999


In [8]:
# можно также посчитать для np.array (уточнить параметры в документации, не выдает табличных значений)
a = np.array([0.7972,  0.0767,  0.4383,  0.7866, 0.8091, 0.1954,  0.6307,  0.6599,  0.1065,  0.0508])

st.zscore(a)

# and for Pandas DF
#data.apply(stats.zscore)

array([ 1.12724554, -1.2469956 , -0.05542642,  1.09231569,  1.16645923,
       -0.8558472 ,  0.57858329,  0.67480514, -1.14879659, -1.33234306])

### Поиск t-score по площади и площади по t-score

In [25]:
# поиск площади слева от t-score (первый параметр - t-score, второй - df, степени свободы, объем выборки - 1)
print(st.t.sf(3.078, 1))

# площади справа и между двумя t-score считаются аналогично z-score

# поиск t-score по площади
# допустим выборка n = 15, тогда степеней свободы 14
df = 14
print(st.t.ppf(0.95, df))

0.09999038172554342
2.1447866879169273


## Confidence intervals

Доверительный интервал для среднего значения (вариант 1). Нормальное распределение генеральной совокупности или выборка (n) > 30. **СКО генеральной совокупности известно**. Используем z-score.

In [20]:
# размер выборки и уровень доверия задаются исследователем
# СКО ген. совокупности для этого метода должно быть известно заранее

# confidence level
conf = 0.95

# размер выборки
n = 35

# стандартное отклонение ген. совокупности
stdev_population = 15

# считаем среднее по выборке и например получаем 7
mean = 7

# тогда интервал считается как среднее +- погрешность (Margin of error, ME, E)
# погрешность считается как E = z-score * stdev / sqrt(n)

# сначала найдем z-score
# так как мы рассматриваем площадь слева и справа от среднего, то для уровня доверия 0.95 нужно посчитать z-score
# либо для 0.025 и поменять знак, либо для 0.975 (удобнее второе) - ниже даны два одинаковых варианта
print(st.norm.ppf(0.975))
print(st.norm.ppf((1+0.95)/2))

1.959963984540054
1.959963984540054


In [26]:
# считаем погрешность
z = st.norm.ppf((1+0.95)/2)
print(z)
E = z * stdev_population / math.sqrt(n)
print(E) # это и есть наше отклонение от среднего при уровне доверия 95%

# строим интервал
print(mean - E, mean + E)

1.959963984540054
4.969415701946049
2.0305842980539506 11.96941570194605


In [27]:
# этот результат означает, что с 95% вероятностью можно утверждать, что истинное среднее ген. совокупности находится
# в этом интервале

# другая интерпретация: если взять 100 выборок, то можно ожидать, что 95 из них будут содержать истинное среднее

**Минимальный размер выборки**

In [ ]:
# зная Margin of error (погрешность) и желаемый уровень доверия можно посчитать
# минимальный размер выборки для z-score и t-score (выводится из формулы погрешности (ME))

In [ ]:
# через z-score (известно СКО ген. совокупности)
conf = 0.95
z = st.norm.ppf((1 + conf)/2 # two-tailed z-score
n = ((z * stdev_population) / E) ** 2

# через t-score
# минимальный размер выборки посчитать нельзя, потому что для определения t-score уже нужно знать размер выборки
# (от этого зависят степени свободы)

In [55]:
# пример из ДЗ

var = 225 # Variance
sigma = math.sqrt(var) # SD
Z = round(st.norm.ppf((1+0.95)/2), 2)  # two-tailed test z-score 95%: 1.96 
error = 3 # Погрешность (ME)

N = ((sigma*Z)/error)**2
math.ceil(N) # Округляем в большую сторону до ближайшего целого числа, чтобы обеспечить мин. выборку

97

Доверительный интервал для среднего значения (вариант 2). Нормальное распределение генеральной совокупности или выборка (n) > 30. **СКО генеральной совокупности НЕизвестно**. Используем t-score.

In [3]:
conf = 0.95
n = 35
mean = 7
stdev_sample = 15 # СКО выборки, в отличие от СКО ген. совокупности известно всегда

df = n - 1 # степени свободы, здесь 35 - 1 = 34
t = st.t.ppf((1 + conf)/2, df) # t-score для two-tailed test
print(t)

# погрешность
E = t * stdev_sample / math.sqrt(n)
print(E)

# интервал
print(mean - E, mean + E)

2.032244509317718
5.152680281095608
1.8473197189043917 12.15268028109561


In [ ]:
# интерпретация интервала аналогична
# обратите внимание, t-distribution дает гораздо более широкий интервал (более консертавтивная оценка)
# при аналогичных параметрах среднего, выборки и СКО (правда здесь для выборки, а не ген. совокупности)

Доверительный интервал для пропорции.

In [8]:
# вариант 1. Расчет руками
successes = 85
trials = 100

p_hat = 85 / 100 # proprortion mean
n = 670 # sample size

conf = 0.95
z = st.norm.ppf((1 + conf) / 2) # two-tailed z-score 1.96

E = z * math.sqrt((p_hat * (1 - p_hat)) / n) # marfin of error, погрешность

print(p_hat - E, p_hat + E)

0.8229625467164996 0.8770374532835004


In [17]:
# вариант 2. Формула из коробки
from statsmodels.stats.proportion import proportion_confint
proportion_confint(670 * 0.85, 670)

(0.8229625467164996, 0.8770374532835004)

## Hypothesis testing (one sample)

### Mean. Population standard deviation known. Z-test

Тестирование гипотезы для среднего значения генеральной совокупности. СКО ген. совокупности известно. Ген. совокупность либо нормально распределена, либо выборка > 30.

Условие примера: считается, что средний возраст попугая 68 лет. Вы как исследователь с этим не согласны. Тогда:

- Нулевая гипотеза: средний возраст попугая 68 лет
- Альтернативная гипотеза: средний возраст попугая НЕ 68 лет (two-tailed test)

При этом известен СКО ген. совокупности (на практике этот параметр никогда не бывает известен, поэтому используется t-test).

In [35]:
# Вариант 1. Расчет вручную.

alpha = 0.05 # уровень значимости, пороговое значение, задается исследователем
conf = 1 - alpha # уровень доверия 0.95

population_mean = 68
population_sd = 13

n = 19
sample_mean = 65

# critical regions criterion for 0.025 and 0.975
# если z_calc будет < z_crit_lower и z_calc > z_crit_upper, нулевую гипотезу можно отвергнуть

z_critical_lower = st.norm.ppf((1 - 0.95) / 2) # должно получиться -1.96
z_crit_upper = st.norm.ppf((1 + 0.95) / 2) # должно получиться 1.96

z_score =  (sample_mean - population_mean) / (population_sd / math.sqrt(n))
print(z_score)

p_value = 2 * st.norm.cdf(z_score) # в случае двустороннего теста z-score нужно умножить на 2 для получения p-value
print(p_value) # вероятность получить среднее 65 при условии, что нулевая гипотеза верна

-1.005899756201694
0.3144637935537208


In [32]:
# интерпретация результата
# способ 1: так как -1.006 в пределах границ -1.96 и 1.96, то отвергнуться нулевую гипотезу мы не можем
# способ 2: так как p-value > alpha, аналогичный вывод

In [33]:
# Вариант 2. Решение из коробки

# имитируем выборку, 19 единиц, среднее 65, СКО 13
data = st.norm.rvs(loc=65,scale=13,size=19)
print(data)

from statsmodels.stats.weightstats import ztest
res = ztest(data, value = 68, alternative = "two-sided") # alternative can also be "smaller" or "larger"
print(res)

[84.63215093 70.39581054 58.23910232 50.66408623 63.34727839 83.85106866
 59.22781682 60.05499827 63.61058378 60.35889768 74.11603621 62.35292485
 61.59836383 67.55513994 74.89254117 56.89876157 83.7565853  55.47046599
 61.17762314]
(-0.9084222671849591, 0.3636551655865079)


### Mean. Population standard deviation unknown. T-test

Тестирование гипотезы для среднего значения генеральной совокупности. СКО ген. совокупности неизвестно. Ген. совокупность либо нормально распределена, либо выборка > 30.

Условие примера: считается, что высота растения составляет 15 дюймов. Вы как исследователь с этим не согласны. Тогда:

- Нулевая гипотеза: средний высота растения 15 дюймов
- Альтернативная гипотеза: средняя высота НЕ 15 дюймов (two-tailed test)

При этом СКО ген. совокупности неизвестно

In [54]:
# Вариант 1. Расчет руками

data = [14, 14, 16, 13, 12, 17, 15, 14, 15, 13, 15, 14]

conf = 0.95
alpha = 1 - conf

population_mean = 15

sample_stdev = np.std(data)
sample_mean = sum(data) / len(data)

n = len(data)
df = n - 1

t_critical_lower = st.t.ppf(((1 - conf) / 2), df)
t_critical_upper = st.t.ppf(((1 + conf) / 2), df)
print(t_critical_lower, t_critical_upper)

t_score =  (sample_mean - population_mean) / (sample_stdev / math.sqrt(n))
print(t_score)

p_value = 2 * st.t.sf(-t_score, df)
print(p_value) # почему корректный p-value появляется только, когда мы ставим "-" перед t-score?

-2.200985160082949 2.200985160082949
-1.7597653802562379
0.10619043895728149


In [43]:
# Вариант 2. Решение из коробки

data = [14, 14, 16, 13, 12, 17, 15, 14, 15, 13, 15, 14]
st.ttest_1samp(a = data, popmean = 15)

Ttest_1sampResult(statistic=-1.6848470783484626, pvalue=0.12014460742498101)

In [53]:
# интерпретация результата
# test statistic within the critical region boundaries, p-value > 0.05: fail to reject H[0]

## Hypothesis testing (two samples)

### T-test. Independent samples

Ниже даны три варианта расчета: 
1. Расчет на основе данных
1. Расчет на основе summary statistics
1. Расчет руками

In [56]:
from __future__ import print_function

import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr

np.random.seed(1)

# Create sample data.
a = np.random.randn(40)
b = 4*np.random.randn(50)

# Use scipy.stats.ttest_ind.
t, p = ttest_ind(a, b, equal_var=False)
print("ttest_ind:            t = %g  p = %g" % (t, p))

# Compute the descriptive statistics of a and b.
abar = a.mean()
avar = a.var(ddof=1)
na = a.size
adof = na - 1

bbar = b.mean()
bvar = b.var(ddof=1)
nb = b.size
bdof = nb - 1

# Use scipy.stats.ttest_ind_from_stats.
t2, p2 = ttest_ind_from_stats(abar, np.sqrt(avar), na,
                              bbar, np.sqrt(bvar), nb,
                              equal_var=False)
print("ttest_ind_from_stats: t = %g  p = %g" % (t2, p2))

# Use the formulas directly.
tf = (abar - bbar) / np.sqrt(avar/na + bvar/nb) # t-score
dof = (avar/na + bvar/nb)**2 / (avar**2/(na**2*adof) + bvar**2/(nb**2*bdof))
pf = 2*stdtr(dof, -np.abs(tf)) # p-value (why minus?)

print("formula:              t = %g  p = %g" % (tf, pf))

ttest_ind:            t = -1.5827  p = 0.118873
ttest_ind_from_stats: t = -1.5827  p = 0.118873
formula:              t = -1.5827  p = 0.118873


### T-test. Dependent samples
Равна ли нулю средняя разница между двумя наборами наблюдений?

In [58]:
np.random.seed(11)
before=st.norm.rvs(scale=30,loc=250,size=100)
after=before+st.norm.rvs(scale=5,loc=-1.25,size=100)
weight_df=pd.DataFrame({"weight_before":before,
                         "weight_after":after,
                         "weight_change":after-before})
weight_df.describe()

,weight_before,weight_after,weight_change
count,100.000000,100.000000,100.000000
mean,250.345546,249.115171,-1.230375
std,28.132539,28.422183,4.783696
min,170.400443,165.913930,-11.495286
25%,230.421042,229.148236,-4.046211
50%,250.830805,251.134089,-1.413463
75%,270.637145,268.927258,1.738673
max,314.700233,316.720357,9.759282


In [60]:
st.ttest_rel(a=before,b=after)
# Итак, мы видим, что у нас есть только 1% шансов найти такие огромные различия между образцами.

Ttest_relResult(statistic=2.5720175998568284, pvalue=0.011596444318439857)